In [1]:
import numpy as np
from numba import jit
from scipy.stats import chi2

n = 20
df = 1
M = 100000
alpha = 0.05
CI = np.zeros((n, 3))

@jit(nopython=True, parallel=True)
def bootstrap_t_func(sample, sample_est, n, M):
    bootstrap_t_est = np.zeros(M)
    for i in range(M):
        bootstrap_sample = np.random.choice(sample, n, replace=True)
        bootstrap_est = np.mean(bootstrap_sample)
        bootstrap_sd_est = np.std(bootstrap_sample, ddof=1)
        bootstrap_t_est[i] = np.sqrt(n) * (bootstrap_est - sample_est) / bootstrap_sd_est
    return bootstrap_t_est

for i in range(n):
    sample = np.random.chisquare(df, n)
    mu_est = np.mean(sample)
    sd_est = np.std(sample, ddof=1)
    bootstrap_est = bootstrap_t_func(sample, mu_est, n, M)
    quants = np.quantile(bootstrap_est, [1-alpha/2, alpha/2])
    CI[i, 0:2] = mu_est - quants * sd_est / np.sqrt(n)
    CI[i, 2] = (CI[i, 0] <= df) & (CI[i, 1] >= df)

coverage_prob = np.sum(CI[:, 2]) / n
print(coverage_prob)


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<function std at 0x000001DE4DFF8CC0>) found for signature:
 
 >>> std(array(float64, 1d, C), ddof=Literal[int](1))
 
There are 2 candidate implementations:
[1m  - Of which 2 did not match due to:
  Overload in function 'array_std': File: numba\np\arraymath.py: Line 467.
    With argument(s): '(array(float64, 1d, C), ddof=int64)':[0m
[1m   Rejected as the implementation raised a specific error:
     TypingError: [1mgot an unexpected keyword argument 'ddof'[0m[0m
  raised from c:\Users\klaus\anaconda3\Lib\site-packages\numba\core\typing\templates.py:784
[0m
[0m[1mDuring: resolving callee type: Function(<function std at 0x000001DE4DFF8CC0>)[0m
[0m[1mDuring: typing of call at C:\Users\klaus\AppData\Local\Temp\ipykernel_14820\217391346.py (17)
[0m
[1m
File "..\..\..\..\..\..\AppData\Local\Temp\ipykernel_14820\217391346.py", line 17:[0m
[1m<source missing

In [11]:
import numpy as np
from numba import jit, prange

n = 10000
df = 1
M = 100000
alpha = 0.05
CI = np.zeros((n, 3))

@jit(nopython=True, parallel=True)
def bootstrap_t_func(sample, sample_est, n, M):
    bootstrap_t_est = np.zeros(M)
    for i in prange(M):
        bootstrap_sample = np.random.choice(sample, n, replace=True)
        bootstrap_est = np.mean(bootstrap_sample)
        # Manually calculate standard deviation with ddof=1
        bootstrap_sd_est = np.sqrt(np.sum((bootstrap_sample - bootstrap_est)**2) / (n - 1))
        bootstrap_t_est[i] = np.sqrt(n) * (bootstrap_est - sample_est) / bootstrap_sd_est
    return bootstrap_t_est

for i in range(n):
    sample = np.random.chisquare(df, n)
    mu_est = np.mean(sample)
    sd_est = np.std(sample, ddof=1)
    bootstrap_est = bootstrap_t_func(sample, mu_est, n, M)
    quants = np.quantile(bootstrap_est, [1-alpha/2, alpha/2])
    CI[i, 0:2] = mu_est - quants * sd_est / np.sqrt(n)
    CI[i, 2] = (CI[i, 0] <= df) & (CI[i, 1] >= df)

coverage_prob = np.sum(CI[:, 2]) / n
print(coverage_prob)


KeyboardInterrupt: 

In [8]:
import numpy as np
from numba import jit, prange

n = 20
df = 1
M = 100000
alpha = 0.05
CI = np.zeros((n, 3))

@jit(nopython=True, parallel=True)
def bootstrap_t_func(sample, sample_est, n, M):
    bootstrap_t_est = np.zeros(M)
    for i in prange(M):
        bootstrap_sample = np.random.choice(sample, n, replace=True)
        bootstrap_est = np.mean(bootstrap_sample)
        bootstrap_sd_est = np.sqrt(np.sum((bootstrap_sample - bootstrap_est)**2) / (n - 1))
        bootstrap_t_est[i] = np.sqrt(n) * (bootstrap_est - sample_est) / bootstrap_sd_est
    return bootstrap_t_est

@jit(nopython=True, parallel=True)
def calculate_CI(n, df, M, alpha):
    CI = np.zeros((n, 3))
    for i in prange(n):
        sample = np.random.chisquare(df, n)
        mu_est = np.mean(sample)
        sd_est = np.sqrt(np.sum((sample - mu_est)**2) / (n - 1))
        bootstrap_est = bootstrap_t_func(sample, mu_est, n, M)
        quants = np.quantile(bootstrap_est, [1 - alpha/2, alpha/2])
        CI[i, 0:2] = mu_est - quants * sd_est / np.sqrt(n)
        CI[i, 2] = (CI[i, 0] <= df) & (CI[i, 1] >= df)
    return CI

CI = calculate_CI(n, df, M, alpha)
coverage_prob = np.sum(CI[:, 2]) / n
print(coverage_prob)


0.9


In [ ]:
import numpy as np
from numba import jit, njit, prange

n = 100000
df = 1
M = 100000
alpha = 0.05
CI = np.zeros((n, 3))

@njit(parallel=True)
def bootstrap_t_func(sample, sample_est, n, M):
    bootstrap_t_est = np.zeros(M)
    for i in prange(M):
        bootstrap_sample = np.random.choice(sample, n, replace=True)
        bootstrap_est = np.mean(bootstrap_sample)
        bootstrap_sd_est = np.sqrt(np.sum((bootstrap_sample - bootstrap_est)**2) / (n - 1))
        bootstrap_t_est[i] = np.sqrt(n) * (bootstrap_est - sample_est) / bootstrap_sd_est
    return bootstrap_t_est

@njit(parallel=True)
def calculate_CI(n, df, M, alpha):
    CI = np.zeros((n, 3))
    for i in prange(n):
        sample = np.random.chisquare(df, n)
        mu_est = np.mean(sample)
        sd_est = np.sqrt(np.sum((sample - mu_est)**2) / (n - 1))
        bootstrap_est = bootstrap_t_func(sample, mu_est, n, M)
        quants = np.quantile(bootstrap_est, [1 - alpha/2, alpha/2])
        CI[i, 0:2] = mu_est - quants * sd_est / np.sqrt(n)
        CI[i, 2] = (CI[i, 0] <= df) & (CI[i, 1] >= df)
    return CI

CI = calculate_CI(n, df, M, alpha)
coverage_prob = np.sum(CI[:, 2]) / n
print(coverage_prob)


0.9
